In [2]:
import pandas as pd
import numpy as np
import os

In [2]:
import csv

In [7]:

with open('metrics_all_data.csv', mode='w', newline='') as test_file:
	file_writer = csv.writer(test_file)
	file_writer.writerow(
		[
            'segmentation', 'F1', 'precision', 'recall', 'accuracy', 'specificity',
            'train_F1', 'train_precision', 'train_recall', 'train_accuracy', 'train_specificity',
        ]
	)


In [3]:


with open('segmented-data-v1-4.csv', mode='w', newline='') as test_file:
	file_writer = csv.writer(test_file)
	file_writer.writerow(
		[
			'img_type', 'patient_id', 'cycle_id', 'slice_id', 'mask_int_mean',
			'confirmed_cancer', 'unconfirmed_cancer', 'benign', 'segment',
		]
	)



In [11]:

path_processed = './processed/'
metadata = pd.read_csv(f'{path_processed}metadata.csv')
(
	metadata
		[metadata.patient.apply(lambda x: x not in [102, 129, 54, 61, 65, 67, 68, 84, 88, 89, 94, 95])]
		.query('has_prostate')
		[['patient', 'cycle', 'slice']]
		.rename({'patient': 'patient_id', 'cycle': 'cycle_id', 'slice': 'slice_id'}, axis='columns')
		.to_csv('./inpsecting.csv', index=False)
)

In [8]:
pd.concat(
	[
		pd.read_csv(
			file,
			dtype={
				'img_type': int,
				'patient_id': int,
				'cycle_id': int,
				'slice_id': int,
				'label': str,
				'confirmed_cancer': str,
				'unconfirmed_cancer': str,
				'benign': str,
				'mask_int_mean': float,
				'segment': int,
			},
		)
		for file in [
			'filtered/segmented-data.csv',
			'filtered/segmented-data-1.csv',
			'filtered/segmented-data-2.csv',
			'filtered/segmented-data-3.csv',
			'filtered/segmented-data-4.csv',
			'filtered/segmented-data-41.csv',
			'filtered/segmented-data-5.csv',
		]
	]
).query('segment == 0 and patient_id == 116').label.value_counts()

False    25844
True        26
Name: label, dtype: int64

In [12]:
for file in [
	'segmented-data-v1-0.csv',
	'segmented-data-v1-1.csv',
	'segmented-data-v1-2.csv',
	'segmented-data-v1-3.csv',
]:
	print(file)
	(
		pd.read_csv('./inpsecting.csv')
			.merge(
				pd.read_csv(
					file,
					dtype={
						'img_type': int,
						'patient_id': int,
						'cycle_id': int,
						'slice_id': int,
						'confirmed_cancer': str,
						'unconfirmed_cancer': str,
						'benign': str,
						'mask_int_mean': float,
						'segment': int,
					},
				),
				how='left',
			)
			.query('img_type != img_type')
			[['patient_id', 'cycle_id', 'slice_id']]
			.to_csv('./inpsecting.csv', index=False)
	)

segmented-data-v1-0.csv
segmented-data-v1-1.csv
segmented-data-v1-2.csv
segmented-data-v1-3.csv


In [59]:
(
	pd.read_csv('./inpsecting.csv')
		[['patient_id', 'slice_id']]
		.drop_duplicates()
		.sort_values(['patient_id', 'slice_id'])
		.to_csv('./inpsecting-1.csv', index=False)
)

In [10]:
pd.read_csv(
	'segmentations/segmentation-6.csv',
	dtype={
		'img_type': int,
		'patient_id': int,
		'cycle_id': int,
		'slice_id': int,
		'label': bool,
		# 'confirmed_cancer': bool,
		# 'unconfirmed_cancer': bool,
		# 'benign': bool,
		'mask_int_mean': float,
		'segment': int,
	},
).query('patient_id == 2').drop(['cycle_id', 'mask_int_mean'], axis='columns').drop_duplicates().label.value_counts()
# [['benign', 'unconfirmed_cancer', 'confirmed_cancer']].drop_duplicates()

False    814
True       5
Name: label, dtype: int64

In [13]:
for file in [
	'segmented-data-v1-0.csv',
	'segmented-data-v1-1.csv',
	'segmented-data-v1-2.csv',
	'segmented-data-v1-3.csv',
	# 'segmented-data.csv',
	# 'segmented-data-1.csv',
	# 'segmented-data-2.csv',
	# 'segmented-data-3.csv',
	# 'segmented-data-4.csv',
	# 'segmented-data-41.csv',
	# 'segmented-data-5.csv',
]:
	print(file)
	(
		pd.read_csv(
			file,
			dtype={
				'img_type': int,
				'patient_id': int,
				'cycle_id': int,
				'slice_id': int,
				# 'confirmed_cancer': str,
				# 'unconfirmed_cancer': str,
				# 'benign': str,
				'confirmed_cancer': bool,
				'unconfirmed_cancer': bool,
				'benign': bool,
				'mask_int_mean': float,
				'segment': int,
			},
		)
			.query('(confirmed_cancer or benign) and mask_int_mean == mask_int_mean')
			.assign(label=lambda x: x.confirmed_cancer.astype(int))
			# .query('(confirmed_cancer == 1.0 or confirmed_cancer == "1.0" or confirmed_cancer == "True" or benign == 1.0 or benign == "1.0" or benign == "True") and mask_int_mean == mask_int_mean')
			# .assign(label=lambda x: x.confirmed_cancer.astype(str).str.match('1.0|True'))
			[[
				'img_type', 'patient_id', 'cycle_id', 'slice_id', 'mask_int_mean',
				'segment', 'label',
			]]
			.to_csv(f'filtered/{file}', index=False)
	)

segmented-data-v1-0.csv
segmented-data-v1-1.csv
segmented-data-v1-2.csv
segmented-data-v1-3.csv


In [132]:

segmentations = 25
shift_up_to = 5
path_processed = './processed/'


def get_non_null(row, columns=['cycle', 'lag', 'lead']):
	values = [row[col] for col in columns if pd.notnull(row[col])]
	if len(values) != 0:
		return values[0]


patients = pd.read_csv(f'{path_processed}metadata.csv')
exclude_patients = np.concatenate(
	[
		patients
			.groupby('patient').cycle.max().reset_index()
			.query('cycle < 10')['patient'],
		[102, 129, 54, 61, 65, 67, 68, 84, 89, 94, 95],
	]
)
patients = (
	patients[patients.patient.apply(lambda x: x not in exclude_patients)]
		.query('has_prostate')
		[['patient', 'cycle']]
		.drop_duplicates()
)

patients['segment'] = (
	patients.cycle
		/ patients.groupby('patient').cycle.transform('max')
		* (segmentations - 1)
).round()

patients = (
	pd.DataFrame(
		np.transpose(
			[
				np.linspace(0, segmentations - 1, segmentations),
				np.repeat([1], segmentations)
			]
		),
		columns=['segment', 'key'],
	)
	.merge(
		pd.DataFrame(
			np.transpose(
				[
					patients.patient.drop_duplicates().sort_values(),
					np.repeat([1], patients.patient.drop_duplicates().shape[0])
				]
			),
			columns=['patient', 'key'],
		)
	)
	.merge(
		patients.groupby(['patient', 'segment']).cycle.max().reset_index(),
		how='outer',
	)
	.sort_values(['patient', 'segment'])
	[['segment', 'patient', 'cycle']]
)

for shift_to in np.linspace(1, shift_up_to, shift_up_to):
	if patients.cycle.notnull().all():
		break
	print(shift_to)
	patients['lag'] = patients.groupby('patient').cycle.shift(shift_to)
	patients['lead'] = patients.groupby('patient').cycle.shift(-shift_to)
	patients['cycle'] = patients.apply(get_non_null, axis='columns')

(
	patients[['segment', 'patient', 'cycle']]
		.rename(
			{'segment': 'new_segment', 'cycle':'segment', 'patient': 'patient_id'},
			axis='columns',
		)
		.to_csv('segmentation_mapping.csv', index=False)
)

1.0
2.0
3.0
4.0


In [14]:
def split_segmentations(file, mapping):
	segmentations = (
		pd.read_csv(
			f'filtered/{file}',
			dtype={
				'img_type': int,
				'patient_id': int,
				'cycle_id': int,
				'slice_id': int,
				'label': bool,
				'mask_int_mean': float,
				'segment': int,
			},
		)
			.merge(mapping)
			.assign(segment=lambda x: x.new_segment)
			.drop(['new_segment'], axis='columns')
	)
	for segmentation_id in mapping.new_segment.drop_duplicates().astype(int):
		if not os.path.exists(f'segmentations/segmentation-{segmentation_id}.csv'):
			(
				segmentations.query(f'segment == {segmentation_id}')
					.drop(['segment'], axis='columns')
					.drop_duplicates()
					.to_csv(f'segmentations/segmentation-{segmentation_id}.csv', index=False)
			)
			continue
		(
			segmentations.query(f'segment == {segmentation_id}')
				.drop(['segment'], axis='columns')
				.drop_duplicates()
				.to_csv(
					f'segmentations/segmentation-{segmentation_id}.csv',
					mode='a', header=False, index=False,
				)
		)


mapping = pd.read_csv('segmentation_mapping.csv')

for file in [
	'segmented-data-v1-0.csv',
	'segmented-data-v1-1.csv',
	'segmented-data-v1-2.csv',
	'segmented-data-v1-3.csv',
	# 'segmented-data.csv',
	# 'segmented-data-1.csv',
	# 'segmented-data-2.csv',
	# 'segmented-data-3.csv',
	# 'segmented-data-4.csv',
	# 'segmented-data-41.csv',
	# 'segmented-data-5.csv',
]:
	print(file)
	split_segmentations(file, mapping)

segmented-data-v1-0.csv
segmented-data-v1-1.csv
segmented-data-v1-2.csv
segmented-data-v1-3.csv
